<a href="https://colab.research.google.com/github/salvadorhm/colab_classify_bot/blob/master/classify_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar Telegram API

In [0]:
!pip install python-telegram-bot==5.3.0

# Configurar Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Seleccionar directorio de trabajo

In [0]:
%cd 'drive/My Drive/classify_bot'
!pwd
!ls

# Librerias

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import sys
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging
import requests

print("Librerías importadas correctamente")

# Classify

In [0]:
def classify(image_path):
    # Read the image_data
    image_data = tf.io.gfile.GFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line in tf.io.gfile.GFile("tf_files/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.io.gfile.GFile("tf_files/retrained_graph.pb", 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.compat.v1.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        
        predictions = sess.run(softmax_tensor, \
                {'DecodeJpeg/contents:0': image_data})
        
        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        val =1
        result = None
        for node_id in top_k:
            human_string = label_lines[node_id]
            accuracy = predictions[0][node_id]
            print('%s (score = %.5f)' % (human_string, accuracy))
            if accuracy >= 0.80:
                #result = human_string + ',tiene un %'+str(accuracy)+'de probabilidad de que sea.'
                result = human_string
                val = 0
        if val == 1:
            result = "grado0"
            #result = "Estas sano"
    return result

print("Método para clasificar imagenes creado correctamente")

# Token de telegram

In [0]:
# ClassifyImagesBot

token = 'xxxxx'
print("token registrado")

# Configuración del bot

In [0]:
# Enable logging
try:
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info('Este mensaje representa algo normal')
except Exception as e:
    print("Error logging {}".format(e.args))

def start(bot, update):
    try:
        username = update.message.from_user.username
        message = "Hola " + username +"  puedo ayudarte?"
        update.message.reply_text(message)
        update.message.reply_text('Puedes usar los comandos /inf, /help')
    except Exception as e:
        print("Error start {}".format(e.args))
def information(bot , update):
    try:
        username = update.message.from_user.username
        message = "Hola " + username +"Este bot fue creado para orientarte en caso de una quemadura"
        update.message.reply_text(message)
        update.message.reply_text('De ser necesario aqui estan los numeros de Protección Civil :')
        update.message.reply_text('Para cuautepec 7542761')
        update.message.reply_text('Para Tulancingo 7757531014 ó 7757530412')
    except Exception as e:
        print("Error start {}".format(e.args))

def help(bot, update):
    try:
        username = update.message.from_user.username
        update.message.reply_text('Hola, para poder auxiliarte en que puedes hacer en tu situacion, solo tienes que mandar una foto de la piel afectada')
    except Exception as e:
        print("Error help {}".format(e.args))

def analize(bot, update):
    try:
        message = "Recibiendo imágen..."
        update.message.reply_text(message)
        print(message)
        
        photo_file = bot.getFile(update.message.photo[-1].file_id)
        id_user = update.message.from_user.id
        id_file = photo_file.file_id
        id_analisis = str(id_user) + "-" + str(id_file)
        filename = os.path.join('downloads/', update.message.from_user['first_name'] +  update.message.from_user['last_name']+'{}.jpg'.format(id_analisis))
        photo_file.download(filename)
        message = "Imagen recibida, analizando, por favor espera un poco"
        update.message.reply_text(message)
        print(message)
        result = classify(filename)
        if result == 'grado0':
            update.message.reply_text("Tranquilo, Estas sano")
        elif result == "grado1":
            update.message.reply_text('La quemadura es de primer grado :')
            update.message.reply_text('|SOLO SIGUE LOS PASOS Y ESTARAS BIEN|')
            update.message.reply_text('1. Enfría la zona afectada')
            update.message.reply_text("Sumerge inmediatamente el sitio de la quemadura en agua fría del grifo o aplica compresas frías y húmedas. Continúa por aproximadamente 10 minutos o hasta que el dolor se alivie.")
            update.message.reply_text('2. Usa vaselina')
            update.message.reply_text('Aplica vaselina dos o tres veces al día. No apliques ungüentos, pasta de dientes o mantequilla a la quemadura, ya que pueden causar una infección. No apliques antibióticos tópicos.')
            update.message.reply_text('3. Cubre con un vendaje')
            update.message.reply_text('Cubre la quemadura con un vendaje antiadherente y estéril. Si se forman ampollas, no las revientes. Déjalas sanar por sí mismas mientras mantienes el área cubierta.')
        elif result == "grado2":
            update.message.reply_text('La quemadura es de segundo grado :')
            update.message.reply_text('|SOLO SIGUE LOS PASOS Y ESTARAS BIEN, SI TE SIENTES MAL ACUDE AL MEDICO|')
            update.message.reply_text('1. Enjuagar la quemadura')
            update.message.reply_text('Enjuague la piel quemada con agua fría hasta que pare el dolor. Generalmente, el enjuague detendrá el dolor al cabo de 15 a 30 minutos. El agua fría baja la temperatura de la piel y evita que la quemadura se agrave. Usted puede:')
            update.message.reply_text('Poner los brazos, las manos, los dedos de las manos, las piernas, los pies o los dedos de los pies en un cuenco de agua fría.')
            update.message.reply_text('No use hielo ni agua helada, los cuales pueden causar daño en los tejidos.')
            update.message.reply_text('2. Limpiar la quemadura')
            update.message.reply_text('Lávese las manos antes de limpiar una quemadura. No toque la quemadura con las manos ni nada sucio, porque las ampollas abiertas pueden infectarse fácilmente.')
            update.message.reply_text('No reviente las ampollas.')
            update.message.reply_text('Lave suavemente la zona quemada con agua limpia. Algo de la piel quemada podría desprenderse con el lavado. Seque la zona con toquecitos suaves usando un paño limpio o gasa.')
        elif result == "grado3":
            update.message.reply_text('La quemadura es de tercer grado :')
            update.message.reply_text('Solo se puede ir con un medico, para tratar con medicamentos o de ser necesario una  cirugía')
            update.message.reply_text('De ser necesario aqui estan los numeros de Protección Civil :')
            update.message.reply_text('Para cuautepec 7542761')
            update.message.reply_text('Para Tulancingo 7757531014 ó 7757530412')
        #print(result)
        print("envio imagen: " + update.message.from_user['first_name'] +  update.message.from_user['last_name'])
        #update.message.reply_text(result)
        print("Waiting image..")
    except Exception as e:
        print("Error analize {}".format(e.args))
def echo(bot, update):
    try:
        update.message.reply_text(update.message.text)
        print("Receiving text...")
        print("Waiting for another test...")
        print(update.message.from_user)
    except Exception as e:
        print("Error echo {}".format(e.args))

def error(bot, update, error):
    try:
        logger.warn('Update "%s" caused error "%s"' % (update, error))
    except Exception as e:
        print("Error error {}".format(e.args))
def main():
    try:
        print('ClassifyImagesBot init token')
        updater = Updater(token)
        #helping()
        # Get the dispatcher to register handlers
        dp = updater.dispatcher
        print('ClassifyImagesBot init dispatcher')
        # on different commands - answer in Telegram
        dp.add_handler(CommandHandler("start", start))
        dp.add_handler(CommandHandler("help", help))
        dp.add_handler(CommandHandler("inf", information))


        # on noncommand detect the document type on Telegram
        dp.add_handler(MessageHandler(Filters.text, echo))
        dp.add_handler(MessageHandler(Filters.photo, analize))

        # log all errors
        dp.add_error_handler(error)

        # Start the Bot
        updater.start_polling()
        print('ClassifyImagesBot ready')
        updater.idle()
        messages = "Hello..."
        pdate.message.reply_text(messages)
        print(messages)        
    except Exception as e:
        print("Error main {}".format(e.message))

print("Bot configurado correctamente")


# Ejecutar el Bot

In [0]:
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print("Error name: {}".format(e.args))